In [2]:
import os
import zipfile
import mlflow
import click
import sklearn
import logging
import tempfile
import keras
import numpy as np
import pandas as pd

from mlflow.keras import log_model, save_model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2

/home/guess/anaconda3/lib/python3.7/site-packages/h5py/_hl/base.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (Mapping, MutableMapping, KeysView,
Using TensorFlow backend.
/home/guess/anaconda3/lib/python3.7/site-packages/tensorflow/python/util/nest.py:824: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  _pywrap_tensorflow.RegisterType("Sequence", _collections.Sequence)
/home/guess/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/checkpointable/util.py:448: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class _ObjectIdentitySet(collections.MutableSet):


In [8]:
df.dtypes

id                     int64
clump_thickness        int64
size_uniformity        int64
shape_uniformity       int64
marginal_adhesion      int64
epithelial_size        int64
bare_nucleoli        float64
bland_chromatin        int64
normal_nucleoli        int64
mitoses                int64
class                  int64
dtype: object

In [3]:

import click
import os
import tempfile
import six
import mlflow

from mlflow.utils import mlflow_tags
from mlflow.entities import RunStatus
from mlflow.utils.logging_utils import eprint
from mlflow.tracking.fluent import _get_experiment_id


def _already_ran(entry_point_name, parameters, git_commit, experiment_id=None):
    """Best-effort detection of if a run with the given entrypoint name,
    parameters, and experiment id already ran. The run must have completed
    successfully and have at least the parameters provided.
    """
    experiment_id = experiment_id if experiment_id is not None else _get_experiment_id()
    client = mlflow.tracking.MlflowClient()
    all_run_infos = reversed(client.list_run_infos(experiment_id))
    for run_info in all_run_infos:
        full_run = client.get_run(run_info.run_id)
        tags = full_run.data.tags
        if tags.get(mlflow_tags.MLFLOW_PROJECT_ENTRY_POINT, None) != entry_point_name:
            continue
        match_failed = False
        for param_key, param_value in six.iteritems(parameters):
            run_value = full_run.data.params.get(param_key)
            if run_value != param_value:
                match_failed = True
                break
        if match_failed:
            continue

        if run_info.status != RunStatus.FINISHED:
            eprint(("Run matched, but is not FINISHED, so skipping "
                    "(run_id=%s, status=%s)") % (run_info.run_id, run_info.status))
            continue

        previous_version = tags.get(mlflow_tags.MLFLOW_GIT_COMMIT, None)
        if git_commit != previous_version:
            eprint(("Run matched, but has a different source version, so skipping "
                    "(found=%s, expected=%s)") % previous_version, git_commit)
            continue
        return client.get_run(run_info.run_id)
    eprint("No matching run has been found.")
    return None


# TODO(aaron): This is not great because it doesn't account for:
# - changes in code
# - changes in dependant steps
def _get_or_run(entrypoint, parameters, git_commit, use_cache=True):
    existing_run = _already_ran(entrypoint, parameters, git_commit)
    if use_cache and existing_run:
        print("Found existing run for entrypoint=%s and parameters=%s" % (entrypoint, parameters))
        return existing_run
    print("Launching new run for entrypoint=%s and parameters=%s" % (entrypoint, parameters))
    submitted_run = mlflow.run(".", entrypoint, parameters=parameters, use_conda=False)
    return mlflow.tracking.MlflowClient().get_run(submitted_run.run_id)



with mlflow.start_run(run_name='single_workflow') as active_run:
#         os.environ['SPARK_CONF_DIR'] = os.path.abspath('.')
#         source_version = active_run.info.source_version
    git_commit = active_run.data.tags.get(mlflow_tags.MLFLOW_GIT_COMMIT)
    gathering = _get_or_run("gathering", {}, git_commit)
#         ratings_csv_uri = os.path.join(load_raw_data_run.info.artifact_uri, "ratings-csv-dir")
#     prep_model_path = os.path.join(gathering.info.artifact_uri, "gathered_data.csv")
    preprocessing = _get_or_run("preprocessing", {}, git_commit)

#     source_model_path = os.path.join(preprocessing.info.artifact_uri, "preprocessed_data.csv")
    modeling = _get_or_run("modeling", {}, git_commit)
#     modeling = _get_or_run("modeling", {'preprocessed_data':source_model_path}, git_commit)






Run matched, but is not FINISHED, so skipping (run_id=19e100eaaa3d43acb087354069404936, status=FINISHED)
Run matched, but is not FINISHED, so skipping (run_id=6d5d47e6e19c47f99dd6f7f8e1c7f0ce, status=FINISHED)
No matching run has been found.
2019/10/23 21:16:43 INFO mlflow.projects: === Created directory /tmp/tmpyh9kysaw for downloading remote URIs passed to arguments of type 'path' ===
2019/10/23 21:16:43 INFO mlflow.projects: === Running command 'python gathering.py' in run with ID 'cf830250b5fe4cf5980b7d7b995e0136' === 


Launching new run for entrypoint=gathering and parameters={}


2019/10/23 21:16:44 INFO mlflow.projects: === Run (ID 'cf830250b5fe4cf5980b7d7b995e0136') succeeded ===
Run matched, but is not FINISHED, so skipping (run_id=5111b0281f8943188a1cf44259fdec24, status=FINISHED)
Run matched, but is not FINISHED, so skipping (run_id=1fdab5dce0fc4ab2af250ee58e2ca8b1, status=FINISHED)
No matching run has been found.
2019/10/23 21:16:44 INFO mlflow.projects: === Created directory /tmp/tmp4mjpbee_ for downloading remote URIs passed to arguments of type 'path' ===
2019/10/23 21:16:44 INFO mlflow.projects: === Running command 'python preprocessing.py' in run with ID 'ff5f4ce9902d479a95e43129f1b7a533' === 


Launching new run for entrypoint=preprocessing and parameters={}


2019/10/23 21:16:45 INFO mlflow.projects: === Run (ID 'ff5f4ce9902d479a95e43129f1b7a533') succeeded ===
Run matched, but is not FINISHED, so skipping (run_id=fdbb8a99885f4f2ba3ab1f8e6ebd05d5, status=FAILED)
Run matched, but is not FINISHED, so skipping (run_id=51bcddd56f544a5d93d993ef443e3b9a, status=FINISHED)
No matching run has been found.
2019/10/23 21:16:46 INFO mlflow.projects: === Created directory /tmp/tmp94hitiii for downloading remote URIs passed to arguments of type 'path' ===
2019/10/23 21:16:46 INFO mlflow.projects: === Running command 'python modeling.py' in run with ID '5ae43703d30c41e99b2c4fe9bb5990a0' === 


Launching new run for entrypoint=modeling and parameters={}


2019/10/23 21:16:47 INFO mlflow.projects: === Run (ID '5ae43703d30c41e99b2c4fe9bb5990a0') succeeded ===


In [6]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df.head()

,A,B,C,D
0,36,82,89,25
1,75,18,61,9
2,74,23,26,81
3,37,51,10,99
4,53,40,73,75


In [9]:
input_data = {
    'columns': list(df.columns),
    'data': df.values.tolist()
}
input_data

{'columns': ['A', 'B', 'C', 'D'],
 'data': [[36, 82, 89, 25],
  [75, 18, 61, 9],
  [74, 23, 26, 81],
  [37, 51, 10, 99],
  [53, 40, 73, 75],
  [1, 90, 35, 59],
  [22, 3, 42, 79],
  [81, 14, 14, 49],
  [70, 56, 59, 91],
  [20, 54, 25, 35],
  [69, 77, 88, 91],
  [43, 21, 57, 50],
  [24, 83, 50, 6],
  [19, 81, 11, 9],
  [59, 45, 69, 43],
  [55, 83, 91, 20],
  [23, 40, 70, 76],
  [23, 66, 62, 28],
  [88, 52, 97, 90],
  [75, 76, 64, 44],
  [46, 77, 60, 87],
  [72, 68, 55, 28],
  [70, 81, 81, 65],
  [86, 59, 42, 96],
  [69, 20, 79, 6],
  [90, 32, 66, 89],
  [4, 53, 94, 44],
  [1, 76, 14, 39],
  [93, 6, 11, 35],
  [90, 69, 31, 27],
  [20, 76, 43, 94],
  [92, 80, 67, 6],
  [77, 46, 83, 14],
  [33, 41, 66, 23],
  [4, 41, 46, 72],
  [15, 39, 81, 63],
  [90, 20, 62, 54],
  [55, 56, 19, 48],
  [95, 46, 1, 39],
  [9, 13, 87, 49],
  [85, 17, 85, 28],
  [61, 40, 22, 91],
  [39, 12, 14, 56],
  [29, 23, 42, 84],
  [2, 68, 34, 8],
  [85, 78, 75, 26],
  [53, 64, 23, 64],
  [57, 54, 45, 36],
  [79, 22, 9,